## pdf 텍스트로 변환 및 요약 코드

In [2]:
import pdfplumber
import requests

# 📂 PDF 파일 경로
pdf_path = "04-Image filtering.pdf"

# 📝 텍스트 누적 저장 변수
full_text = ""

# 1️⃣ PDF에서 텍스트 추출
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            full_text += f"\n--- [페이지 {i+1}] ---\n{text}"

# 2️⃣ 추출된 텍스트 저장 (원문)
with open("output_text.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print("✅ PDF에서 텍스트 추출 완료 및 저장됨.")

import os
from dotenv import load_dotenv
import google.generativeai as genai

# ✅ .env 파일에서 환경변수 불러오기
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# ✅ Gemini API 설정
genai.configure(api_key=api_key)

# 모델 호출
model = genai.GenerativeModel("models/gemini-2.5-flash")

# PDF 텍스트 불러오기
with open("output_text.txt", "r", encoding="utf-8") as f:
    content = f.read()

# 프롬프트 설정
prompt = f"""
다음은 이미지 필터링 강의 PDF에서 추출한 텍스트입니다.
핵심 내용을 **한국어로** 요약해 주세요:

{content}
"""

# Gemini API 호출
response = model.generate_content(prompt)

# 출력
print(response.text)

with open("summary.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

✅ PDF에서 텍스트 추출 완료 및 저장됨.


C:\Users\nices\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


다음은 이미지 필터링 강의 PDF의 핵심 내용 요약입니다.

---

**이미지 필터링의 핵심 요약**

이 강의는 이미지 필터링의 주요 개념인 선형 필터링, 스무딩 및 샤프닝, 그리고 필터의 분리성(Separability)에 대해 다룹니다.

**1. 이미지 필터링의 기본 개념**
*   **필터링이란?** 각 출력 픽셀의 값이 해당 입력 픽셀 및 주변 픽셀 값에 의존하는 "주변 영역 연산(Neighborhood operation)"을 의미합니다.

**2. 선형 필터 (Linear Filters)**
*   **정의:** 이미지 `f`와 필터(커널 또는 마스크) `w` 간의 "곱의 합(sum-of-products)" 연산으로, `g(x,y) = ΣΣ w(s,t) f(x+s, y+t)`로 표현됩니다. 커널 `w`의 크기는 연산 영역을, 계수는 필터의 특성을 결정합니다.
*   **직관:** 필터의 패턴과 지역 이미지 패턴이 유사할 때 필터 반응이 최대로 나타납니다.
*   **계산 비용:** `m x m` 크기의 커널과 `n x n` 크기의 이미지에 대해 총 `m^2 * n^2`번의 곱셈이 필요합니다.
*   **경계 효과 (Boundary Effect):** 이미지 가장자리에서 필터 창이 이미지 범위를 벗어날 때 이를 처리하는 방법(제로 패딩, 복제, 반사, 순환 등)이 필요합니다.
*   **상관(Correlation) vs. 컨볼루션(Convolution):** 컨볼루션은 커널을 뒤집어(flip) 연산하며, 더 나은 수학적 속성 때문에 주로 사용됩니다.
*   **속성:**
    *   **덧셈성(Additivity) 및 동질성(Homogeneity):** 이 두 가지를 만족하면 선형 연산입니다.
    *   **이동 불변성(Shift Invariance):** 출력 결과가 픽셀의 절대 위치에 관계없이 동일합니다.
*   **CNN과의 연관성:** 컨볼루션 신경망(CNN)의 기본 연산은 학습된 선형 필터(컨볼루션)와 비선형 함수를 결합하여 사용합니다

## 녹음본 텍스트 변환

In [3]:
import whisper
import os
import time
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 사용 디바이스: {device}")

# Whisper는 CPU에서는 fp16을 자동 비활성화함
model = whisper.load_model("small", device=device)

audio_file = "image_filtering.m4a"


if not os.path.exists(audio_file):
    raise FileNotFoundError(f"❌ 음성 파일이 존재하지 않습니다: {audio_file}")

print("🎙️ 음성 인식 시작...")
start_time = time.time()

# ⛔ GPU는 fp16, CPU는 fp32로 자동 설정됨
# 따라서 따로 설정할 필요 없음
result = model.transcribe(audio_file, language="ko")

end_time = time.time()
elapsed_time = end_time - start_time

text = result["text"]
segments = result["segments"]
total_duration = segments[-1]['end'] if segments else 0.0

print("✅ 음성 인식 완료!")
print(f"🕐 총 인식된 음성 길이: {total_duration:.2f}초")
print(f"⚙️ 처리 시간: {elapsed_time:.2f}초")

output_path = "25-3-27-1_transcript.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(text)

print(f"💾 텍스트 저장 완료 → {output_path}")


📌 사용 디바이스: cpu
🎙️ 음성 인식 시작...


C:\Users\nices\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ 음성 인식 완료!
🕐 총 인식된 음성 길이: 2922.60초
⚙️ 처리 시간: 1071.28초
💾 텍스트 저장 완료 → 25-3-27-1_transcript.txt


## 녹음본 요약 코드

In [4]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# ✅ API 설정
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ 모델 선택
model = genai.GenerativeModel("models/gemini-2.5-flash")

# ✅ 녹음된 텍스트 불러오기
with open("25-3-27-1_transcript.txt", "r", encoding="utf-8") as f:
    raw_recording_text = f.read()

# ✅ 프롬프트 설정
prompt = f"""
다음 텍스트는 구어체로 진행된 강의 녹취록입니다.
이 텍스트를 문어체로 정리하여 명확하고 간결한 강의 요약 형식으로 바꿔주세요.

녹음 내용:
\"\"\"{raw_recording_text}\"\"\"
"""

# ✅ Gemini API 호출 및 응답 저장
try:
    response = model.generate_content(prompt)
    if response.candidates and response.candidates[0].content.parts:
        cleaned_text = response.candidates[0].content.parts[0].text.strip()

        with open("recording_summary.txt", "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        print("✅ 녹음 텍스트 정리 완료 → 'recording_summary.txt'로 저장됨.")
    else:
        print("⚠️ Gemini 응답이 비어 있습니다.")
except Exception as e:
    print(f"❌ 오류 발생: {e}")


✅ 녹음 텍스트 정리 완료 → 'recording_summary.txt'로 저장됨.


## 텍스트 강조

In [5]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from tqdm import tqdm

# 🔐 API 설정
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("models/gemini-2.5-flash")

# ✅ PDF 요약 문장들 불러오기
with open("summary.txt", "r", encoding="utf-8") as f:
    summary_sentences = [line.strip() for line in f if line.strip()]

# ✅ 녹음 요약 문장들 불러오기
with open("recording_summary.txt", "r", encoding="utf-8") as f:
    recording_sentences = [line.strip() for line in f if line.strip()]

# ✅ 강조 대상 문장 판단
highlight_set = set()

for r_line in tqdm(recording_sentences, desc="🔍 Gemini로 의미 유사도 평가 중"):
    for s_line in summary_sentences:
        prompt = f"""
다음 두 문장이 의미상 유사한 경우에만 "1"을 출력하세요.  
의미가 다르면 "0"을 출력하세요.  
문장은 순서나 표현이 달라도 의미가 같으면 유사하다고 간주하세요.

문장 A: "{r_line}"  
문장 B: "{s_line}"

숫자 하나만 출력하세요.
"""
        try:
            response = model.generate_content(prompt)
            score = response.text.strip()
            if score == "1":
                highlight_set.add(r_line)
                break  # 하나라도 유사하면 더 비교할 필요 없음
        except:
            continue

# ✅ 강조된 텍스트 생성 및 저장
highlighted_lines = []
for line in recording_sentences:
    if line in highlight_set:
        highlighted_lines.append(f"✅ {line}")
    else:
        highlighted_lines.append(line)

output_path = "recording_summary_highlighted.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(highlighted_lines))

print(f"✅ 의미 기반 강조 완료 → {output_path}")


🔍 Gemini로 의미 유사도 평가 중: 100%|██████████| 39/39 [15:29<00:00, 23.84s/it]

✅ 의미 기반 강조 완료 → recording_summary_highlighted.txt


## 중요도 기반 퀴즈 생성

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# ✅ API 설정
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("models/gemini-1.5-flash")

# ✅ 전체 요약 텍스트 불러오기
with open("recording_summary_highlighted.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# ✅ 프롬프트 정의
prompt = f"""
다음은 강의 요약 텍스트입니다.  
- ✅ 표시된 문장은 반드시 퀴즈 문제에 포함되도록 해주세요.  
- 그 외 문장들은 선택적으로 반영해도 됩니다.
- 문제는 총 5문제로 구성해주세요.
- 문제 유형은 객관식(4지선다), OX, 단답형으로 다양하게 섞어주세요.
- 각 문항에 정답도 함께 작성해주세요.
- 한국어로 출력해주세요.

📄 강의 요약:
\"\"\"{full_text}\"\"\"
"""

# ✅ Gemini 호출
response = model.generate_content(prompt)

# ✅ 결과 저장
with open("quiz_from_highlighted_and_full.txt", "w", encoding="utf-8") as f:
    f.write(response.text.strip())

print("✅ 퀴즈 생성 완료 → quiz_from_highlighted_and_full.txt 저장됨")


C:\Users\nices\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
]